# 3. Regularisierte Regression: LASSO & Ridge

**KI1-Projekt 308** — California Housing Datensatz

**Schwerpunkt P1:** Feature-Selektion via LASSO, Ridge-Regression,
polynomiale Feature-Transformation, Skalierungseffekte.

Vorlage: HA9/10 Aufgabe 3 und HA11 Aufgabe 2

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures

from utils.data import load_and_clean_data, get_train_test_split
from utils.evaluation import evaluate_model, add_result
from utils.plotting import plot_predicted_vs_actual, plot_residuals, save_fig

plt.rcParams['figure.dpi'] = 100
%matplotlib inline

## 3.1 Daten laden (ohne Skalierung)

In [ ]:
df = load_and_clean_data()
X_train, X_test, y_train, y_test, feature_names = get_train_test_split(df)
print(f"Features: {feature_names}")

## 3.2 LassoCV — Feature-Selektion ohne Skalierung

In [ ]:
lasso_cv = LassoCV(cv=5, random_state=42, max_iter=10000)
lasso_cv.fit(X_train, y_train)

print(f"Optimales alpha: {lasso_cv.alpha_:.6f}\n")

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Koeffizient': lasso_cv.coef_
})
print("LASSO Koeffizienten:")
print(coef_df.to_string(index=False))

selected_features = coef_df[coef_df['Koeffizient'] != 0]['Feature'].tolist()
print(f"\nSelektierte Features ({len(selected_features)}/{len(feature_names)}): {selected_features}")

result_lasso = evaluate_model(lasso_cv, X_train, X_test, y_train, y_test, "LassoCV (ohne Skalierung)")
add_result(result_lasso)

## 3.3 Ridge Regression

In [ ]:
ridge_cv = RidgeCV(alphas=np.logspace(-3, 3, 100), cv=5)
ridge_cv.fit(X_train, y_train)

print(f"Optimales alpha: {ridge_cv.alpha_:.6f}\n")

result_ridge = evaluate_model(ridge_cv, X_train, X_test, y_train, y_test, "RidgeCV (ohne Skalierung)")
add_result(result_ridge)

## 3.4 Mit MinMaxScaler

In [ ]:
X_train_mm, X_test_mm, y_train_mm, y_test_mm, scaler_mm, fn = get_train_test_split(df, scaler='minmax')

lasso_cv_mm = LassoCV(cv=5, random_state=42, max_iter=10000)
lasso_cv_mm.fit(X_train_mm, y_train_mm)

print(f"Optimales alpha (MinMax): {lasso_cv_mm.alpha_:.6f}")
print("\nKoeffizienten:")
for name, coef in zip(fn, lasso_cv_mm.coef_):
    print(f"  {name:15s}: {coef:.6f}")

result_lasso_mm = evaluate_model(lasso_cv_mm, X_train_mm, X_test_mm, y_train_mm, y_test_mm, "LassoCV (MinMax)")
add_result(result_lasso_mm)

## 3.5 Mit StandardScaler

In [ ]:
X_train_ss, X_test_ss, y_train_ss, y_test_ss, scaler_ss, fn = get_train_test_split(df, scaler='standard')

lasso_cv_ss = LassoCV(cv=5, random_state=42, max_iter=10000)
lasso_cv_ss.fit(X_train_ss, y_train_ss)

print(f"Optimales alpha (Standard): {lasso_cv_ss.alpha_:.6f}")
print("\nKoeffizienten:")
for name, coef in zip(fn, lasso_cv_ss.coef_):
    print(f"  {name:15s}: {coef:.6f}")

result_lasso_ss = evaluate_model(lasso_cv_ss, X_train_ss, X_test_ss, y_train_ss, y_test_ss, "LassoCV (Standard)")
add_result(result_lasso_ss)

## 3.6 Polynomiale Feature-Transformation + LASSO

In [ ]:
# Polynomiale Features (Grad 2) auf skalierten Daten
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_ss)
X_test_poly = poly.transform(X_test_ss)

print(f"Originale Features: {X_train_ss.shape[1]}")
print(f"Polynomiale Features: {X_train_poly.shape[1]}")

lasso_poly = LassoCV(cv=5, random_state=42, max_iter=10000)
lasso_poly.fit(X_train_poly, y_train_ss)

result_lasso_poly = evaluate_model(
    lasso_poly, X_train_poly, X_test_poly, y_train_ss, y_test_ss,
    "LassoCV (Poly Grad 2 + Standard)"
)
add_result(result_lasso_poly)

In [ ]:
# Vergleichsplot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_predicted_vs_actual(
    y_test_ss, lasso_cv_ss.predict(X_test_ss),
    title="LassoCV (Standard)", ax=axes[0]
)
plot_predicted_vs_actual(
    y_test_ss, lasso_poly.predict(X_test_poly),
    title="LassoCV (Poly+Standard)", ax=axes[1]
)

fig.suptitle("Vergleich: Linear vs. Polynomial", fontsize=14)
fig.tight_layout()
save_fig(fig, "lasso_comparison")
plt.show()

## 3.7 Zusammenfassung

| Erkenntnis | Detail |
|-----------|--------|
| Feature-Selektion via LASSO | Identifiziert die wichtigsten Features |
| Skalierung | Nötig für polynomiale Transformation |
| Ridge vs. LASSO | Ridge behält alle Features, LASSO selektiert |
| Polynomiale Features | Leichte Verbesserung durch Interaktionsterme |